In [5]:
from extra_deps.fast_labeling_workflow.fast_labeling_utils import visualize, train_model
from src.annotation_handling.segmentsai_handler_old import (setup_connection_segmentsai,
                                                visualize_dataset,
                                                create_dataset, 
                                                upload_images_to_segments,
                                                generate_and_upload_predictions
                                                )
from segments import SegmentsDataset

#### Setup Global Variables

In [6]:
client = setup_connection_segmentsai()
TRAIN_DATASET_NAME = 'etaylor/cannabis_patches_all_images'

#### 1. Upload your images and label a small subset

##### Vizualize dataset images with masks

In [ ]:
# Visualize the dataset
visualize_dataset(client, TRAIN_DATASET_NAME)

#### 2. Train a segmentation model on the labeled images


In [4]:
# Initialize a dataset from the release file
release = client.get_release(TRAIN_DATASET_NAME, 'v0.1')
dataset = SegmentsDataset(release)

Initializing dataset...
Preloading all samples. This may take a while...


100%|██████████| 74/74 [00:00<00:00, 1059.95it/s]

Initialized dataset with 74 images.


In [ ]:
# Train an instance segmentation model on the dataset
model = train_model(dataset)

#### Create a new dataset and upload images to him

In [6]:
# Directory containing your images
image_name = "IMG_2145"
week = "week9_3xzoom_regular"

name = f"trichomes_patches_{week}_{image_name}"
description = f"Trichomes patches week {week} of image {image_name}."
task_type = "segmentation-bitmap"
task_attributes = {
    "format_version": "0.1",
    "categories": [
        {"name": "trichome", "id": 1},
    ]
}

absolute_images_path = f"/home/etaylor/images/images_patches/cannabis_patches/{week}/{image_name}"
sharpness_absolute_images_path = f"/home/etaylor/images/images_patches/cannabis_patches/testing_methods/preprocess_by_sharpness/{week}/{image_name}"

dataset_name_test = f"etaylor/{name}"


In [7]:
#Create the dataset:
dataset_test = create_dataset(client, name, description, task_type, task_attributes)
print(dataset_test)

name='trichomes_patches_week9_3xzoom_regular_IMG_2145' full_name='etaylor/trichomes_patches_week9_3xzoom_regular_IMG_2145' cloned_from=None description='Trichomes patches week week9_3xzoom_regular of image IMG_2145.' category='other' public=False owner=Owner(username='etaylor', created_at='2022-12-28T12:53:18Z', email=None) created_at='2023-10-04T15:29:52.497701Z' enable_ratings=False enable_skip_labeling=True enable_skip_reviewing=False enable_save_button=False enable_label_status_verified=False enable_same_dimensions_track_constraint=False enable_interpolation=True task_type='segmentation-bitmap' label_stats=LabelStats(TOTAL=None, LABELED=None, UNLABELED=None, PRELABELED=None) samples_count=0 collaborators_count=None task_attributes=TaskAttributes(format_version='0.1', categories=[TaskAttributeCategory(name='trichome', id=1, color=None, has_instances=None, attributes=None, dimensions=None)], image_attributes=None) labelsets=None role=None readme='' metadata={} noncollaborator_can_lab

### Upload the Images that are not annotaed to the dataset

In [ ]:
upload_images_to_segments(client, dataset_name_test, absolute_images_path)

#### Create a new release to the new dataset

In [9]:
name = 'v0.1'
description = 'upload predictions to datset.'
release = client.add_release(dataset_name_test, name, description)
# if the release is already existing, you can get it with:
# release = client.get_release(dataset_name_test, name)
print(release)

uuid='c90f2bbe-e2c6-416b-9324-94aaf2ba5fb3' name='v0.1' description='upload predictions to datset.' release_type='JSON' attributes=URL(url='') status='PENDING' created_at='2023-10-04T15:30:17.664729Z' samples_count=21


In [ ]:
generate_and_upload_predictions(client, dataset_name_test, model, visualize_flag=True)

In [ ]:
# Delete the dataset - ONLY IF YOU WANT TO DELETE THE DATASET!!!
client.delete_dataset("etaylor/trichomes_patches_week8_zoom3_regular_IMG_IMG_1809")